![banner_etl](https://github.com/cistelsa/Commerce_Data_Analysis_and_Recommendations/blob/main/5_Sources/Images/banner_automatizacion.gif?raw=true)

**<mark style="background:#2bfe9c">Script de automatización</mark> proviene del Notebook  <mark>ETL_hb_hotels</mark>**

In [19]:
# Usaremos librería Pandas y Json para permitir la lectura de los archivos
import pandas as pd
import json

StatementMeta(, c54f4e55-6189-4a49-a8ca-c73c9a732674, 21, Finished, Available)

In [20]:
# Ruta base donde se encuentran los archivos JSON
path_base = "/lakehouse/default/Files/data/original/HotelBeds/"

StatementMeta(, c54f4e55-6189-4a49-a8ca-c73c9a732674, 22, Finished, Available)

### Realizamos lectura del dataset

In [21]:
# Hacemos lectura del dataset en csv previamente extraido.
df_hotels_c = pd.read_json(path_base + "hotels_dataset.json")

StatementMeta(, c54f4e55-6189-4a49-a8ca-c73c9a732674, 23, Finished, Available)

In [22]:
# Revisamos su estructura, tipos de datos
df_hotels_c.info()

StatementMeta(, c54f4e55-6189-4a49-a8ca-c73c9a732674, 24, Finished, Available)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39963 entries, 0 to 39962
Data columns (total 31 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   code                   39963 non-null  int64 
 1   name                   39963 non-null  object
 2   description            39923 non-null  object
 3   countryCode            39963 non-null  object
 4   stateCode              39963 non-null  object
 5   destinationCode        39963 non-null  object
 6   zoneCode               39963 non-null  int64 
 7   coordinates            39963 non-null  object
 8   categoryCode           39963 non-null  object
 9   categoryGroupCode      39876 non-null  object
 10  chainCode              33551 non-null  object
 11  accommodationTypeCode  39963 non-null  object
 12  boardCodes             38280 non-null  object
 13  segmentCodes           39166 non-null  object
 14  address                39963 non-null  object
 15  postalCode         

Analizamos columnas como `license` revisamos el diccionario de datos y es una licencia interna de Hotelbeds, vemos que los campos vacíos están en más del 99% por lo cual se considera eliminar la columna, tampoco es relevante para los KPIs y los objetivos.

In [23]:
df_hotels_c.head()
pd.options.display.max_columns=0

StatementMeta(, c54f4e55-6189-4a49-a8ca-c73c9a732674, 25, Finished, Available)

La **lista** unicamente son referencia de código que hace un llamado a otra tabla relacionada es el caso de `boardCodes` y `segmentCodes`

La **Lista de Diccionarios** hace un llamado a más parametros o descripciones de servicios que tiene el Hotel, estos se pueden manejar en su mayoría como una tabla relacionada, este es un proceso de ETL y normalizado de tablas.

Podemos observar que las columnas que son en forma de **Diccionarios** son: `name`, `description`, `coordinates`, `address`, `city` estas columnas se procede a desanidar para que quede en formato de string

In [24]:
# Define una función para extraer el contenido de 'content'
def extract_content(json_dict):
    if isinstance(json_dict, dict):
        return json_dict.get('content', '')
    else:
        return ''

StatementMeta(, c54f4e55-6189-4a49-a8ca-c73c9a732674, 26, Finished, Available)

In [25]:
# Aplica la función a la columna y crea una nueva columna con el contenido extraído
df_hotels_c['name'] = df_hotels_c['name'].apply(extract_content)
df_hotels_c['description'] = df_hotels_c['description'].apply(extract_content)
df_hotels_c['address'] = df_hotels_c['address'].apply(extract_content)
df_hotels_c['city'] = df_hotels_c['city'].apply(extract_content)

StatementMeta(, c54f4e55-6189-4a49-a8ca-c73c9a732674, 27, Finished, Available)

In [26]:
# Separamos coordinates en dos columnas respectivamente longitude y latitude
def extract_longitude(json_dict):
    if isinstance(json_dict, dict):
        return json_dict.get('longitude', '')
    else:
        return ''
    
def extract_latitude(json_dict):
    if isinstance(json_dict, dict):
        return json_dict.get('latitude', '')
    else:
        return ''

StatementMeta(, c54f4e55-6189-4a49-a8ca-c73c9a732674, 28, Finished, Available)

In [27]:
# Aplicamos la Función
df_hotels_c['longitude'] = df_hotels_c['coordinates'].apply(extract_longitude)
df_hotels_c['latitude'] = df_hotels_c['coordinates'].apply(extract_latitude)

StatementMeta(, c54f4e55-6189-4a49-a8ca-c73c9a732674, 29, Finished, Available)

Eliminamos algunas columnas que ya se encuentran en el dataset de `hotels_details_dataset.csv`, es el caso de `terminals, issues, wildcards, images, interestPoints, facilities, rooms` en otros casos como el de `license` por tener el 99% de campos vacíos se procede a eliminar la columna ya verificando que los datos no son relevantes, `coordinates` se elimina ya que se dividión en dos columnas.

In [28]:
# Eliminar columnas con datos null: 'terminals', 'license', 'issues', 'wildcards', 'images', 'interestPoints', 'facilities', 'rooms'
df_hotels_c = df_hotels_c.drop(columns=['terminals', 'license', 'issues', 'wildcards', 'images', 'interestPoints', 'facilities', 'rooms'])
# Eliminar Columna ya gestionada y dividida
df_hotels_c = df_hotels_c.drop(columns=['coordinates'])

StatementMeta(, c54f4e55-6189-4a49-a8ca-c73c9a732674, 30, Finished, Available)

In [29]:
df_hotels_c.info()

StatementMeta(, c54f4e55-6189-4a49-a8ca-c73c9a732674, 31, Finished, Available)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39963 entries, 0 to 39962
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   code                   39963 non-null  int64  
 1   name                   39963 non-null  object 
 2   description            39963 non-null  object 
 3   countryCode            39963 non-null  object 
 4   stateCode              39963 non-null  object 
 5   destinationCode        39963 non-null  object 
 6   zoneCode               39963 non-null  int64  
 7   categoryCode           39963 non-null  object 
 8   categoryGroupCode      39876 non-null  object 
 9   chainCode              33551 non-null  object 
 10  accommodationTypeCode  39963 non-null  object 
 11  boardCodes             38280 non-null  object 
 12  segmentCodes           39166 non-null  object 
 13  address                39963 non-null  object 
 14  postalCode             39904 non-null  object 
 15  ci

In [30]:
# Cambiar el nombre a columna code
df_hotels_c = df_hotels_c.rename(columns={
    "code":"hotel_id",
    "countryCode": "country_code",
    "stateCode": "state_code",
    "destinationCode": "destination_code",
    "zoneCode": "zone_code",
    "categoryCode": "category_code",
    "categoryGroupCode": "categorygroup_code",
    "chainCode": "chain_code",
    "accommodationTypeCode": "accommodationtype_code"
})

StatementMeta(, c54f4e55-6189-4a49-a8ca-c73c9a732674, 32, Finished, Available)

In [31]:
# eliminar boardCodes segmentCodes ya que están relacionados en su propia tabla
df_hotels_c = df_hotels_c.drop(['boardCodes', 'segmentCodes'], axis=1)

StatementMeta(, c54f4e55-6189-4a49-a8ca-c73c9a732674, 33, Finished, Available)

In [32]:
#Reemplazando los caracteres especiales de la columna "S2C"
df_hotels_c['S2C'] = df_hotels_c['S2C'].str.replace('*','')

StatementMeta(, c54f4e55-6189-4a49-a8ca-c73c9a732674, 34, Finished, Available)

/tmp/ipykernel_8355/2251670612.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_hotels_c['S2C'] = df_hotels_c['S2C'].str.replace('*','')


In [33]:
#Cambiando el tipo de dato de la columna "S2C"
df_hotels_c['S2C']= df_hotels_c['S2C'].astype('float')

StatementMeta(, c54f4e55-6189-4a49-a8ca-c73c9a732674, 35, Finished, Available)

In [34]:
df_city_code = pd.DataFrame()
df_city_code['city_name'] = df_hotels_c['city']
df_city_code['state_code'] = df_hotels_c['state_code']
# Eliminamos duplicados
df_city_code.drop_duplicates(subset=['city_name'], inplace=True)
df_city_code['city_code'] = range(100, 100 + len(df_city_code))
# Organizar columnas
df_city_code = df_city_code[['city_code', 'state_code', 'city_name']]
df_city_code.info()

StatementMeta(, c54f4e55-6189-4a49-a8ca-c73c9a732674, 36, Finished, Available)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8990 entries, 0 to 39961
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   city_code   8990 non-null   int64 
 1   state_code  8990 non-null   object
 2   city_name   8990 non-null   object
dtypes: int64(1), object(2)
memory usage: 280.9+ KB


In [35]:
#Importante Creación de dataset cities.csv
df_city_code.to_csv("/lakehouse/default/Files/data/launch/HotelBeds/cities.csv", index=False)

StatementMeta(, c54f4e55-6189-4a49-a8ca-c73c9a732674, 37, Finished, Available)

In [36]:
# Inner Join para crear la columna city_code en el dataset
df_hotels_c = df_hotels_c.merge(df_city_code[['city_code', 'city_name']], left_on="city", right_on="city_name", how="inner")

StatementMeta(, c54f4e55-6189-4a49-a8ca-c73c9a732674, 38, Finished, Available)

In [37]:
# eliminar city y city_name ya que están relacionados en su propia tabla
df_hotels_c = df_hotels_c.drop(['city', 'city_name'], axis=1)

StatementMeta(, c54f4e55-6189-4a49-a8ca-c73c9a732674, 39, Finished, Available)

In [38]:
df_hotels_c.info()

StatementMeta(, c54f4e55-6189-4a49-a8ca-c73c9a732674, 40, Finished, Available)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 39963 entries, 0 to 39962
Data columns (total 22 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   hotel_id                39963 non-null  int64  
 1   name                    39963 non-null  object 
 2   description             39963 non-null  object 
 3   country_code            39963 non-null  object 
 4   state_code              39963 non-null  object 
 5   destination_code        39963 non-null  object 
 6   zone_code               39963 non-null  int64  
 7   category_code           39963 non-null  object 
 8   categorygroup_code      39876 non-null  object 
 9   chain_code              33551 non-null  object 
 10  accommodationtype_code  39963 non-null  object 
 11  address                 39963 non-null  object 
 12  postalCode              39904 non-null  object 
 13  email                   20233 non-null  object 
 14  phones                  39505 non-null

In [39]:
#Generando dataset hotels_hb_dataset.csv
df_hotels_c.to_csv("/lakehouse/default/Files/data/beta/Hotelbeds/hotels_hb_dataset.csv", index=False)

StatementMeta(, c54f4e55-6189-4a49-a8ca-c73c9a732674, 41, Finished, Available)